# <span style='color:black'> Import lib </span>

In [1]:
from shutil import copyfile
from tkinter import *
from PIL import *
from Model.Funtion_Bank import *
from Model.DL_Network_Model import Net
#from Model.Funtion_Bank import data_input

import os.path
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
import PIL.Image
#import random
import time
#from random import randint
import itertools


global btn_txt_matrix
global btn_matrix
global btn_refresh_matrix
global Button
global Current_play_ground

# <span style='color:black'> Define Agent </span>

In [2]:
import numpy as np

class agent_a:
    def __init__(self, perception, state):
        self.perception = perception
        self.state = state

        #print(self.verify_Matrix.argmax())
    def restart(self):
        print(self.input_string)
        return('abc')
    def next_action(self):
        print('next action')


# <span style='color:black'> define file_path and parameters </span>

In [28]:
input_file_path = 'Data/training_data_input.csv'
score_file_path = 'Data/training_data_score.csv'
next_action_file_path = 'Data/training_data_next_action_taken.csv'

file_name_model_latest_version = 'Model/model_latest_version.pt'
file_name_model_last_version = 'Model/model_last_version.pt'
torch.set_default_tensor_type('torch.cuda.FloatTensor')

# <span style='color:black'> Game engine </span>

In [4]:
def play_ground(row, column, wall_generated, target_generated, agent_a, agent_b):
    rows = row * 3 + 1
    columns = column * 3 + 1
    init_ground = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.float)
    wall_generated.append([row, column, row, column * 2])
    wall_generated.append([row, column, row * 2, column])
    wall_generated.append([row * 2, column, row * 2, column * 2])
    wall_generated.append([row, column * 2, row * 2, column * 2])
    for line in wall_generated:
        for i in range(line[0], line[2] + 1):
            for j in range(line[1], line[3] + 1):
                init_ground[i][j] = 1
    for point in target_generated:
        init_ground[point[0]][point[1]] = 2
    for point in agent_a:
        init_ground[point[0]][point[1]] = 3
    for point in agent_b:
        init_ground[point[0]][point[1]] = 4
    return init_ground

In [29]:
def data_input(file_path):
    #to explore: remove duplicatioin - if both input and score are same, then remove duplication
    data_original = pd.read_csv(file_path, header = None)
    #data_original = data_original.drop_duplicates(subset=None, keep='first', inplace=False)
    torch_tensor = torch.tensor(data_original.values, dtype=torch.float)
    #print(torch_tensor)
    return torch_tensor

def DL_training_model():
    indicator = 0
    learning_rate = 0.1
    epoch_size = 10000
    steps_for_printing_out_loss = 1000
    #cropped_perspective = np.array([4,5,6])
    input_data = data_input(input_file_path)
    score_data = data_input(score_file_path)
    next_action_data = data_input(next_action_file_path)
    
    input = input_data
    #print(input)
    target = score_data
    net = Net()
    loss_functioin = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = learning_rate)
    if indicator == 1:
        state_dict_last_version = torch.load('Model/model_latest_version.pt')['state_dict']
        net.load_state_dict(state_dict_last_version)
    for i in range(1,epoch_size + 1):
        optimizer.zero_grad()
        output = net(input)
        output[next_action_data == 0] = 0
        #output[target == -2] = -2
        #output[target == -1.1] = -1.1
        loss = loss_functioin(output, target)
        loss.backward()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
        # Does the update
        optimizer.step()
    
    if indicator == 1:
        copyfile(file_name_model_latest_version, file_name_model_last_version)
    else:
        torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_last_version)
    torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)


def DL_model(cropped_perspective, model_version, state):
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    next_vision = model_latest_version(torch.tensor(cropped_perspective.reshape(651), dtype=torch.float))
    #next_vision[1] = 2
    for i in range(0,4):
        if state[i] == 1:
            next_vision[i] = -3
    #print(next_vision)

    #print(next_vision)
    """
    if ((state[0] == -1) and (state[1] == 1)) or ((state[0] == 0) and (state[1] == playground_w)):
        next_vision[0] = -3
    if ((state[0] == -1) and (state[3] == playground_w)) or ((state[0] == 0) and (state[3] == 1)):
        next_vision[2] = -3
    """
    #print('abc')
    #print(next_vision)
    next_step = next_vision.argmax()
    action = next_step
    return action



In [6]:
"""
1. add a rule of agent movement:
- only 1 agent is allowed at the same location point
    - how to deal with confliction?
        - let all agents to predict the next step
        - list down confliction
        - how to decide, get an agreement?
            - scenario 1: all agents are within same group, and communication channel is available. And agreement can be made based on some rule set
                - what is the logic? 
                    - maybe compare the Q value? 
                    - or to figure out a way to calculate team performance impact on each scenario?
                    - or find a leader to decide?
                    - others?
            - scenario 2: not all agents are within same group, and communication channel is not available partially or completely
                - what is the solution?
                - 
- wall


- each individual location point, to use an array to represent the current statement:
    - 0: None
    - 1: Wall
    - 2: target
    - 3: agent group A
    - 4: agent group B
    - TBC: how to indicate last moment of Turth?
        - maybe to use recurring NN
    - initial multi-agent location generation VS wall
    

- target:
    - maybe assign different score to each target with some logic can be observed by agent? (enhancement)
    - to add a target 
    - target can be moved as well(enhancement)
    - to play the agent A to chase against agent B (enhancement)

- agent:
    - 0-8? or 0-5?
    - visioin control
        - only explored location is visible
        - communication is allowed within certain agent group
    - leader
    - small group of agent within agent group
    - 

- Score:
    - write a game engine:
        - option 1:
            - when to stop
                - within 200 steps, who gets more score, then win.
            - what is the score
                - the difference of target between two groups of agent?
    - DQ learning:
        - any other algorithm?
    
    - driven by comparation or his or her best peerformance? 
        - Like in real world, kids used to be compared with others' performance may be better but there is a risk of aggressive
    
    - individual outstanding achivement
    - others has team reward
    
    - reward shouldl be higher if game can be finished fast - this should be considered as part of DQ?
                
- agent group:
    - for now, it is 2 groups.
    - in future, we can try multi-groups together.

- build a dostributed system to support multi-agent.

- optimize the DL learning model:
    - enhance model archtect
    - tune the parameters

- UI:
    - display result

- result comparation:
    - using python to draw Dashboard and do comparation

"""





#DL_training_model()
#DL_model()




"\n1. add a rule of agent movement:\n- only 1 agent is allowed at the same location point\n    - how to deal with confliction?\n        - let all agents to predict the next step\n        - list down confliction\n        - how to decide, get an agreement?\n            - scenario 1: all agents are within same group, and communication channel is available. And agreement can be made based on some rule set\n                - what is the logic? \n                    - maybe compare the Q value? \n                    - or to figure out a way to calculate team performance impact on each scenario?\n                    - or find a leader to decide?\n                    - others?\n            - scenario 2: not all agents are within same group, and communication channel is not available partially or completely\n                - what is the solution?\n                - \n- wall\n\n\n- each individual location point, to use an array to represent the current statement:\n    - 0: None\n    - 1: Wall\

# <span style='color:black'> Main entrance </span>

In [38]:
def step_movement(i):
    switcher={
    0:[-1, 0],
    1:[0, -1],
    2:[1, 0],
    3:[0, 1],
    4:[0, 0]
    }
    return switcher.get(i,"Invalid next action")

def remove_duplication_in_lists(k):
    if not k:
        return []
    elif len(k) == 1:
        print(k)
        return list(k)
    else:
        res = [] 
        for i in k: 
            if i not in res: 
                res.append(i)
        return res

    

def agent_next_location(play_ground_numpy, agent_current_location, model_version, row, column):
    #centre_point = [15, 23]
    matrix_trancated = trancate_matrix(play_ground_numpy, agent_current_location, row, column)
    #print(matrix_trancated)    #21, 31
    state = []
    for k in range(0, 4):
        location = np.add(agent_current_location, step_movement(k))
        state.append(play_ground_numpy[location[0], location[1]])
    #print('state')
    #print(state)
        
    next_action = DL_model(matrix_trancated,1,state)
    next_action = np.flip(next_action.cpu().numpy())
    #print(step_movement(next_action))
    next_location = np.add(agent_current_location, step_movement(next_action))
    return matrix_trancated.reshape(651), next_action, [next_location[0], next_location[1]]



if __name__ == "__main__":
    DQ_ratio = 0.9
    row = 10
    column = 15
    no_of_wall = 3
    no_of_target = 7
    no_of_agent = 5
    
    #initial position
    wall_generated = wall_generation(row, column, no_of_wall)
    target_generated = target_generation(row, column, no_of_target)
    agent_a = agent_generation(row, column, no_of_agent)
    agent_b = agent_generation(row, column, no_of_agent)
    game_rounds = 30
    agent_A_current_round_score = 0
    agent_B_current_round_score = 0

    agent_A_next_action = [[] for i in range(0, no_of_agent)]
    agent_A_perceptioin = [[] for i in range(0, no_of_agent)]
    agent_A_next_action_array = [[] for i in range(0, no_of_agent)]
    agent_A_score = [[] for i in range(0, no_of_agent)]
    agent_B_next_action = [[] for i in range(0, no_of_agent)]
    agent_B_perceptioin = [[] for i in range(0, no_of_agent)]
    agent_B_next_action_array = [[] for i in range(0, no_of_agent)]
    agent_B_score = [[] for i in range(0, no_of_agent)]

    
    for current_round in range(0, game_rounds):
        tensor_play_ground = play_ground(row, column, wall_generated, target_generated, agent_a, agent_b)
        
        tensor_play_ground_numpy = tensor_play_ground.clone()
        play_ground_numpy = np.flip(tensor_play_ground_numpy.cpu().numpy(), 0)
        
        model_version = 1
        
        agent_A_found_target = []
        agent_B_found_target = []

        
        
        #print(agent_a)
        for agent_count in range(0, no_of_agent):
            agent_a_perception, agent_a_next_action, agent_a[agent_count] = agent_next_location(play_ground_numpy, agent_a[agent_count], model_version, row, column)
            agent_b_perception, agent_b_next_action, agent_b[agent_count] = agent_next_location(play_ground_numpy, agent_b[agent_count], model_version, row, column)

            agent_A_perceptioin[agent_count].append(agent_a_perception)
            agent_A_next_action[agent_count].append(agent_a_next_action)
            action_array = [0] * (no_of_agent)
            action_array[agent_a_next_action] = 1
            agent_A_next_action_array[agent_count].append(action_array)
            agent_A_score[agent_count].append([0] * (no_of_agent))
            
            agent_B_perceptioin[agent_count].append(agent_b_perception)
            agent_B_next_action[agent_count].append(agent_b_next_action)
            action_array = [0] * (no_of_agent)
            action_array[agent_b_next_action] = 1
            agent_B_next_action_array[agent_count].append(action_array)
            agent_B_score[agent_count].append([0] * (no_of_agent))
            
            for item_array in target_generated:
                #print(item_array)
                if np.equal(agent_a[agent_count],item_array).all():
                    agent_A_found_target.append(agent_a[agent_count])
                    break
            for item_array in target_generated:
                if np.equal(agent_b[agent_count],item_array).all():
                    agent_B_found_target.append(agent_b[agent_count])
                    break
        agent_A_found_target = remove_duplication_in_lists(agent_A_found_target)
        agent_B_found_target = remove_duplication_in_lists(agent_B_found_target)
        full_found_target = agent_A_found_target + agent_B_found_target
        full_found_target = remove_duplication_in_lists(agent_A_found_target + agent_B_found_target)
        #print(full_found_target)
        for item in full_found_target:
            target_generated.remove(item)
        agent_A_current_round_score += len(agent_A_found_target)
        agent_B_current_round_score += len(agent_B_found_target)
        total_current_round = current_round + 1
        if not target_generated:
            break
    
    #print(agent_A_score)
    #print(agent_B_score)

    agent_A_final_score = agent_A_current_round_score - agent_B_current_round_score
    agent_B_final_score = agent_A_final_score * -1
    print("agent_A_current_round_score: " + str(agent_A_current_round_score))
    print("agent_B_current_round_score: " + str(agent_B_current_round_score))
    print("agent_A_final_score: " + str(agent_A_final_score))
    print("agent_B_final_score: " + str(agent_B_final_score))
    print("total_current_round: " + str(total_current_round))
    
    for agent_count in range(0, no_of_agent):
        for k in range(0, total_current_round):
            agent_A_score[agent_count][-(k+1)][agent_A_next_action[agent_count][-(k+1)]] = ((DQ_ratio)**(k)) * agent_A_final_score
            agent_B_score[agent_count][-(k+1)][agent_B_next_action[agent_count][-(k+1)]] = ((DQ_ratio)**(k)) * agent_B_final_score
    
    DL_model_input_data = []
    DL_model_output_data = []
    next_action_taken = []
    next_action_taken_array = []
    for agent_count in range(0, no_of_agent):
        DL_model_input_data = DL_model_input_data + agent_A_perceptioin[agent_count]
        DL_model_output_data = DL_model_output_data + agent_A_score[agent_count]
        next_action_taken = next_action_taken + agent_A_next_action[agent_count]
        next_action_taken_array = next_action_taken_array + agent_A_next_action_array[agent_count]
    for agent_count in range(0, no_of_agent):
        DL_model_input_data = DL_model_input_data + agent_B_perceptioin[agent_count]
        DL_model_output_data = DL_model_output_data + agent_B_score[agent_count]
        next_action_taken = next_action_taken + agent_B_next_action[agent_count]
        next_action_taken_array = next_action_taken_array + agent_B_next_action_array[agent_count]

    input_df = pd.DataFrame(DL_model_input_data)
    input_df.to_csv(input_file_path, index=False, mode='a', header=False)
    next_action_taken_df = pd.DataFrame(next_action_taken_array)
    next_action_taken_df.to_csv(next_action_file_path, index=False, mode='a', header=False)
    score_df = pd.DataFrame(DL_model_output_data)
    score_df.to_csv(score_file_path, index=False, mode='a', header=False)
    #print(DL_model_output_data)
    
    
        #print(agent_a)
    #pd.DataFrame(matrix_trancated.reshape(1, 21 * 31)).to_csv(input_file_path, index=False, mode='a', header=False)
    #input_status_df.to_csv(input_file_path, index=False, mode='a', header=False)

[[14, 28]]
[[15, 24]]
[[18, 18]]
[[18, 18]]
[[17, 26]]
[[17, 26]]
[[18, 22]]
[[18, 22]]
agent_A_current_round_score: 4
agent_B_current_round_score: 1
agent_A_final_score: 3
agent_B_final_score: -3
total_current_round: 30


In [39]:
DL_training_model()

Loss (epoch: 1000): 0.13188852
Loss (epoch: 2000): 0.09042186
Loss (epoch: 3000): 0.07135681
Loss (epoch: 4000): 0.06931289
Loss (epoch: 5000): 0.06924354
Loss (epoch: 6000): 0.0688283
Loss (epoch: 7000): 0.06876992
Loss (epoch: 8000): 0.06876029
Loss (epoch: 9000): 0.06874372
Loss (epoch: 10000): 0.06873153


In [9]:
#DL_model(matrix_trancated,1,1)
print(agent_A_found_target)
#DL_model(matrix_trancated,1,1)
print(agent_B_found_target)
print(target_generated)

print(item)
print(full_found_target)
agent_B_found_target

[]
[]
[[15, 18], [13, 26], [13, 28]]
[12, 17]
[]


[]

In [10]:
def remove_duplication_in_lists(k):
    if not k:
        return []
    elif len(k) == 1:
        print(k)
        return list(k)
    else:
        res = [] 
        for i in k: 
            if i not in res: 
                res.append(i)
        return res

k = agent_A_found_target + agent_B_found_target + agent_B_found_target
print(k)
k = remove_duplication_in_lists(k)
print(k)
#return list(k for k,_ in itertools.groupby(k))

[]
[]


In [11]:
print(agent_a)
print(i)
print(agent_B_found_target)
print(agent_A_found_target + agent_B_found_target)
print(play_ground_numpy)
#print(matrix_trancated)
np.savetxt("foo.csv", play_ground_numpy, delimiter=",")

[[19, 16], [17, 18], [19, 16], [19, 16], [19, 16]]


NameError: name 'i' is not defined

In [ ]:
m = remove_duplication_in_lists([[12,3]])
print('d')

In [ ]:
agent_B_score = [[] for i in range(0, 6)]
print(agent_B_score)
agent_A_found_target
len(agent_A_found_target)


agent_A_score[1]

In [ ]:
#leader
#predict other agent next action
#against other agent's prediction
#communicatoin matrix
#memory
#remove some wall
#set up starting point for agent a & b
#speed & count parameter
#use an array to present each point - []
[0,1,0,0,0,0,0,0,0,0,0]
